In [8]:
#------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------     SECCION DE IMPORTS       -----------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------

# Importar BeautifulSoup y requests para poder conectarse a las páginas del SECOP
from bs4 import BeautifulSoup
import requests
import eventlet
eventlet.monkey_patch()

# Importar pyodbc para conectarse a bd de Azure
import pyodbc 

# Importar paquete de descarga de archivos
from tqdm import tqdm

# Importar los paquetes para poder interactuar con carpetas del S.O.
# Y de conexión a Azure

import os
import uuid
import sys
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

#------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------     SECCION DE FUNCIONES     -----------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------

# Crear función que permite establecer los índices de un caracter determinado
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]


# Función para cargar documentos en Azure
def cargar_a_blob():
    try:
        # Crear la conexión al Servicio de Almacenamiento en Azure "hackatonms"
        block_blob_service = BlockBlobService(account_name='hackaton3', account_key='qX3xaSomZIydnBsAGTNfZ/hhLugz9282RQ5qGGNe/KsVUYcgO9//3HvyWJAQlpEeXVFhxIhy+VJEWlTINxkPog==')

        # Definir el contenedor donde se van a cargar todos los documentos
        container_name ='pdf'

        # Volver los permisos del contenedor públicos.
        block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)

        # Buscar los archivos que están en la carpeta definida
        local_path=os.path.abspath("../Documentos Secop/")
        for filename in os.listdir(local_path):
            full_path_to_file =os.path.join(local_path, filename)
            print("Temp file = " + full_path_to_file)
            print("\nUploading to Blob storage as blob" + filename)
            
            # Cargar archivo en el blob storage
            block_blob_service.create_blob_from_path(container_name, filename, full_path_to_file)
            

    except Exception as e:
        print(e)

#------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------  VARIALBES Y PARAMETRIZACION   ----------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------

# Sentencia SQL para escribir en la BD
sentenciaSQL = ""

# Contador para los archivos
j = 0

# Conectarse a la BD hackatonms de Azure

conn = pyodbc.connect(
    r'Driver={ODBC Driver 13 for SQL Server};'
    r'Server=sqldbserverdesafio3.database.windows.net;'
    r'Database=SQLDB;'
    r'UID=laura;'
    r'PWD=Desafio3;')

#------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------     LÓGICA     --------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------

# Sentencia SQL a partir de un cursor
cursor = conn.cursor()
cursor.execute('SELECT DISTINCT top 5 idcontrato, url FROM dbo.contratos order by idcontrato')

# Iteración sobre todos los contratos que se encuentren en la tabla
for row in cursor:
    # Inicializar la variable i en 1 cada vez que se tenga un contrato nuevo
    j = 1
    
    # Id de contrato
    contrato = row[0]
    
    # Enlace de la página a la que se le va a hacer el scrapping
    page_link = row[1]
    
    # Conectarse a la página con el link determinado    
    try:
        with eventlet.Timeout(20):
            page_response = requests.get(page_link, verify=False)
        
        page_content = BeautifulSoup(page_response.content, "html.parser")
        
        # Identificar todos los lugares dentro de la página donde hay información descargable.
        
        # Caso1: De acuerdo con la estructura, el contenedor comienza por "a" y el id comienta por "lnkDocumentDownloadLink"
        descargable = page_content.find_all("a", id=lambda value: value and value.startswith("lnkDocumentDownloadLink"))
        
        for i in range (0,len(descargable)):
            if len(descargable[i]) > 0:
                # Se identifican los índices del caracter ' en la cadena de texto debido a que el substring del mínimo 
                # y máximo de estos va a permitir obtener el link de descarga del documento.
                texto = str(descargable[i])
                minimo = min(findOccurrences(texto,"'"))
                maximo = max(findOccurrences(texto,"'"))
                
                # Substring del texto para obtener solo la parte final de la url
                texto = texto[minimo:maximo]
                
                # Agregarle el inicio de la URL
                texto = 'https://community.secop.gov.co' + texto
                
                # Se eliminan los caracteres que no hacen parte del link
                texto = texto.replace("'","")
                texto = texto.replace("+","")
                texto = texto.replace(" ","")
                
                # Toma la URL de descarga y le aplica la misma metodología para llegar a la URL del PDF
                page_response = requests.get(texto, verify=False)
                texto2 = BeautifulSoup(page_response.content, "html.parser")
                
                minimo = min(findOccurrences(str(texto2),"'"))
                maximo = max(findOccurrences(str(texto2),"'"))
                
                # Substring del texto para obtener solo la parte final de la url
                texto2 = str(texto2)
                texto2 = texto2[minimo:maximo]
                
                # Agregarle el inicio de la URL
                texto2 = 'https://community.secop.gov.co' + texto2
                
                # Se eliminan los caracteres que no hacen parte del link
                texto2 = texto2.replace("'","")
                texto2 = texto2.replace("+","")
                texto2 = texto2.replace(" ","")
                
                # Descargar la información al disco local
                r = requests.get(texto2)
                print("Peso del archivo: " + str(len(r.content)) + " b.")               
                
                # Solicitud HTTP de realizar la descarga
                response = requests.get(texto2, stream=True)
                ruta = "../Documentos SECOP/"
                nombreArchivo = contrato + "_" + str(j)
                
                j = j + 1
                # Agregar a la sentencia SQL el insert del contenido
                sentenciaSQL += "INSERT INTO dbo.contratos_pdf VALUES('" + contrato + "','Documento Contractual','" + texto2 + "','"+ nombreArchivo + ".pdf');"
                
                with open(ruta + nombreArchivo + ".pdf", "wb") as handle:
                    for data in tqdm(response.iter_content()):
                        handle.write(data)
                
                                
        # Caso2: De acuerdo con la estructura, el contenedor comienza por "a" y el id comienta por "lnkDownloadLinkP3Gen"
        descargable = page_content.find_all("a", id=lambda value: value and value.startswith("lnkDownloadLinkP3Gen"))        
        
        for i in range (0,len(descargable)):
            if len(descargable[i]) > 0:
                # Se identifican los índices del caracter ' en la cadena de texto debido a que el substring del mínimo 
                # y máximo de estos va a permitir obtener el link de descarga del documento.
                texto = str(descargable[i])
                minimo = min(findOccurrences(texto,"'"))
                maximo = max(findOccurrences(texto,"'"))
                
                # Substring del texto para obtener solo la parte final de la url
                texto = texto[minimo:maximo]
                
                # Agregarle el inicio de la URL
                texto = 'https://community.secop.gov.co' + texto
                
                # Se eliminan los caracteres que no hacen parte del link
                texto = texto.replace("'","")
                texto = texto.replace("+","")
                texto = texto.replace(" ","")
                                
                # Toma la URL de descarga y le aplica la misma metodología para llegar a la URL del PDF
                page_response = requests.get(texto, verify=False)
                texto2 = BeautifulSoup(page_response.content, "html.parser")
                
                minimo = min(findOccurrences(str(texto2),"'"))
                maximo = max(findOccurrences(str(texto2),"'"))
                
                # Substring del texto para obtener solo la parte final de la url
                texto2 = str(texto2)
                texto2 = texto2[minimo:maximo]
                
                # Agregarle el inicio de la URL
                texto2 = 'https://community.secop.gov.co' + texto2
                
                # Se eliminan los caracteres que no hacen parte del link
                texto2 = texto2.replace("'","")
                texto2 = texto2.replace("+","")
                texto2 = texto2.replace(" ","")
                
                            
                # Descargar la información al disco local
                r = requests.get(texto2)
                print("Peso del archivo: " + str(len(r.content)) + " b.")               
                
                # Solicitud HTTP de realizar la descarga
                response = requests.get(texto2, stream=True)
                ruta = "../Documentos SECOP/"
                nombreArchivo = contrato + "_" + str(j)
                
                j = j + 1
                # Agregar a la sentencia SQL el insert del contenido
                sentenciaSQL += "INSERT INTO dbo.contratos_pdf VALUES('" + contrato + "','Detalle Oportunidad','" + texto2 + "','"+ nombreArchivo + ".pdf');"
                
                with open(ruta + nombreArchivo + ".pdf", "wb") as handle:
                    for data in tqdm(response.iter_content()):
                        handle.write(data)
                
        # Caso3: De acuerdo con la estructura, el contenedor comienza por "a" y el id comienta por "linkDocumentLink"
        descargable = page_content.find_all("a", id=lambda value: value and value.startswith("linkDocumentLink"))        
        
        for i in range (0,len(descargable)):
            if len(descargable[i]) > 0:
                # Se identifican los índices del caracter ' en la cadena de texto debido a que el substring del mínimo 
                # y máximo de estos va a permitir obtener el link de descarga del documento.
                texto = str(descargable[i])
                
                if "iframe" not in texto:
                    minimo = min(findOccurrences(texto,"'"))
                    maximo = max(findOccurrences(texto,"'"))
                    
                    # Substring del texto para obtener solo la parte final de la url
                    texto = texto[minimo:maximo]
                    
                    # Agregarle el inicio de la URL
                    texto = 'https://community.secop.gov.co' + texto
                    
                    # Se eliminan los caracteres que no hacen parte del link
                    texto = texto.replace("'","")
                    texto = texto.replace("+","")
                    texto = texto.replace(" ","")
                    
                    # Toma la URL de descarga y le aplica la misma metodología para llegar a la URL del PDF
                    page_response = requests.get(texto, verify=False)
                    texto2 = BeautifulSoup(page_response.content, "html.parser")
                    
                    minimo = min(findOccurrences(str(texto2),"'"))
                    maximo = max(findOccurrences(str(texto2),"'"))
                    
                    # Substring del texto para obtener solo la parte final de la url
                    texto2 = str(texto2)
                    texto2 = texto2[minimo:maximo]
                    
                    # Agregarle el inicio de la URL
                    texto2 = 'https://community.secop.gov.co' + texto2
                    
                    # Se eliminan los caracteres que no hacen parte del link
                    texto2 = texto2.replace("'","")
                    texto2 = texto2.replace("+","")
                    texto2 = texto2.replace(" ","")
                    
                    # Descargar la información al disco local
                    r = requests.get(texto2)
                    print("Peso del archivo: " + str(len(r.content)) + " b.")               
                    
                    # Solicitud HTTP de realizar la descarga
                    response = requests.get(texto2, stream=True)
                    ruta = "../Documentos SECOP/"
                    nombreArchivo = contrato + "_" + str(j)
                    
                    j = j + 1
                    # Agregar a la sentencia SQL el insert del contenido
                    sentenciaSQL += "INSERT INTO dbo.contratos_pdf VALUES('" + contrato + "','Reporte','" + texto2 + "','"+ nombreArchivo + ".pdf');"
                   
                    with open(ruta + nombreArchivo + ".pdf", "wb") as handle:
                        for data in tqdm(response.iter_content()):
                            handle.write(data)
                                    
    except requests.exceptions.Timeout:
        # No hacer nada si hay una Excepción de timeout 
        print("error en " & page_link )               
        
# Después del loop y web scrapping de todos los contratos de la tabla dbo.contratos_pdf 
# se guardan las url de los PDF que están asociados a esos contratos

##cursor.execute(sentenciaSQL)
##conn.commit()


conn.close()

cargar_a_blob()

C:\Anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Peso del archivo: 8194311 b.


8194311it [02:13, 61446.50it/s]
C:\Anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Peso del archivo: 471942 b.


471942it [00:06, 68785.07it/s]
C:\Anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

Peso del archivo: 10024077 b.


10024077it [03:21, 49708.66it/s]
C:\Anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Peso del archivo: 48884994 b.


29955720it [10:45, 90998.90it/s] 

KeyboardInterrupt: 